# Patchify a VIIRS mask

In [15]:
# imports
import h5py
import numpy as np
from skimage.transform import downscale_local_mean, resize_local_mean
import os

# Load a VIIRS file 

In [5]:
viirs_file = os.path.join(os.getenv('OS_SST'), 'VIIRS',
                          'Extractions', 'VIIRS_2013_85clear_192x192_inpaint.h5')

In [6]:
f = h5py.File(viirs_file, 'r')
f.keys()

<KeysViewHDF5 ['fields', 'inpainted_masks', 'metadata']>

## Check a mask

In [8]:
f['inpainted_masks'][0]

array([[3.4777832, 3.477963 , 3.4783235, ...,       nan,       nan,
              nan],
       [3.4783752, 3.4785554, 3.4789166, ...,       nan,       nan,
              nan],
       [3.4795585, 3.47974  , 3.480102 , ...,       nan,       nan,
              nan],
       ...,
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan]], dtype=float32)

In [13]:
mask = np.isfinite(f['inpainted_masks'][0])
np.sum(mask)/(192**2)

0.13685438368055555

In [14]:
mask.shape

(192, 192)

# Rebin

In [ ]:
r_mask = resize_local_mean(mask, (64, 64))